# Introduction
In this notebook, I will dig deeper into the energy metabolism of our strain and figure out how that works to make sure this is correct, and representative.

Considering ATP is the main currency in the cell, this will be my main point to start investigating. 
Also I will look into the maintanence energy (NGAM) and GAM that is encorporated into the biomass reaction. 

In [2]:
import cameo
import pandas as pd
import cobra.io

In [3]:
#to check how things are done here
model_e_coli = cameo.load_model('iML1515')

In [4]:
model = cobra.io.read_sbml_model('../model/p-thermo.xml')

__Maintanence__
Page 113 of Beata's PhD thesis includes chemostat data that was used to determine the GAM and NGAM of the strain used. she estimates a GAM of 25.44 mmolATP/gDCW/h and a NGAM of 1.53 mmolATP/gDCW/h.

In our model, we currently have two reactions: ATPM and NGAM, both containing the same reaction (hydrolysis of ATP for maintenance). Currently, the ATPM reaction has (-20,20) bounds, where the NGAM reaction is at least irreversible with bounds of (0,20). In the optimization, the NGAM carries no flux, where ATPM is used to generate ATP from ADP, which is ofcourse incorrect. 

Both of these reactions do not reflect the maintenance set in Beata's thesis. The lower bound of this reaction should be set at the identified NGAM so that this flux is somewhat fixed at this minimal value. 

solution: I will remove the NGAM reaction (to stick to ATPM ID), and set this lower bound at the calculated 1.53. Then check the effect on the biomass accumulation rate. 

In [ ]:
model.reactions.ATPM.bounds = (1.53, 1000)

In [ ]:
model.remove_reactions(model.reactions.NGAM)

In [ ]:
model.optimize().objective_value

You can see the biomass accumulation decreases a bit, because there is less ATP generation from the ATPM now. This seems correct, I will save & commit this and push the branch to merge it, with Martyn's review.

In [ ]:
#save&commit
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')

__Growth associated maintenance__
The GAM has been estimated at 25.44 mmolATP/gDCW/h from experiments in Beata's thesis. This should be included in the biomass reaction as extra ATP needed to be consumed to produce 1gDCW/h. 


# Fixing reversibility or other reactions
it seems there may be quite some reactions that are reversible where they should not be. 

Seems to be a cycle between biotin and biotin-5-amp: they just intercovert and in doing so just produced ATP from nothing. So I will check the reversability of these steps. Considering ATP is involved, the BTNLIG reaction should be irreversible.

In [6]:
model.metabolites.actp_c.name = 'Acetyl phosphate'

In [7]:
model.metabolites.btn_c.summary()

,,PERCENT,FLUX,REACTION_STRING
RXN_STAT,ID,,,
PRODUCING,BTNLIG,100.0,1000.0,atp_c + btn_c + h_c <=> b5amp_c + ppi_c
CONSUMING,BTN5AMPL,100.0,1000.0,b5amp_c + h2o_c <=> amp_c + btn_c + 2.0 h_c


In [8]:
model.metabolites.b5amp_c.summary()

,,PERCENT,FLUX,REACTION_STRING
RXN_STAT,ID,,,
PRODUCING,BTN5AMPL,100.0,1000.0,b5amp_c + h2o_c <=> amp_c + btn_c + 2.0 h_c
CONSUMING,BTNLIG,100.0,1000.0,atp_c + btn_c + h_c <=> b5amp_c + ppi_c


In [9]:
model.reactions.BTNLIG.bounds = (0,1000)

Reactions with ATP should generally not be reversible. So I will make a data frame (and export this) to be able to analyze which reactions with ATP are reversible, and if they should be so or not. This can hopefully help me look more into the ATP energetics and resolve some energy generating cycles that we may have.

In [10]:
rev_atp_id = []
rev_atp_reaction = []
for rct in model.metabolites.atp_c.reactions:
    if rct.upper_bound > 0 and rct.lower_bound < 0:
        rev_atp_id.append(rct.id)
        rev_atp_reaction.append(rct.reaction)
    else: 
        continue
        

In [11]:
atp_rev = pd.DataFrame({'ID':rev_atp_id, 'Reaction':rev_atp_reaction})
atp_rev[0:5]

,ID,Reaction
0,XYLUPT,atp_c + xylu__L_c <=> adp_c + xu1p__D_c
1,GLYK,atp_c + glyc_c <=> adp_c + glyc3p_c + h_c
2,R5PPPT,atp_c + r5p_c <=> amp_c + prpp_c
3,CYTK2,atp_c + dcmp_c <=> adp_c + dcdp_c
4,NDPK3,atp_c + cdp_c <=> adp_c + ctp_c + h_c


In [12]:
atp_rev.to_csv('../databases/Rev_ATP_rcts.csv', sep = ',')

In [58]:
#should be irreversible, but then the model doesn't work
#model.reactions.NADK.bounds = (0,1000)
solution['NADK']

In [14]:
#irreversible PFK_3
model.reactions.PFK_3.bounds = (0,1000)

In [15]:
#irreversible PNTK
model.reactions.PNTK.bounds = (0,1000)

In [16]:
#irreversible TMDK1
model.reactions.TMDK1.bounds = (0,1000)

In [17]:
#irreversible URIK1
model.reactions.URIK1.bounds = (0,1000)

In [18]:
#irreversible XYLK
model.reactions.XYLK.bounds = (0,1000)

In [19]:
model.metabolites.r5p_c.name = 'D-Ribose 5-phosphate'
model.metabolites.prpp_c.name = '5-Phospho-alpha-D-ribose 1-diphosphate'

In [20]:
model.reactions.HETZK.bounds = (0,1000)

In [21]:
#GMPS sohould not be reversible
model.reactions.GMPS.bounds = (0,1000)

In [22]:
model.reactions.CYTDK1.bounds = (0,1000)

In [23]:
model.reactions.GLYK.bounds = (0,1000)

In [24]:
model.reactions.PYK.bounds = (0,1000)

In [25]:
#ALDD2x should be irreversible, as right now you get a cycle that produces ATP erroneously
#but fixing this bound kills the models growth.
#model.reactions.ALDD2x.bounds = (0,1000)
solution['ALDD2x']

-1000.0

In [26]:
model.metabolites.acetone_c.name = 'Acetone'

In [27]:
model.reactions.ACTCO2L.bounds = (0,1000)

In [28]:
#based on just the ATP hydrolysis this reaction is probably not reversible
model.reactions.PRAISh.bounds = (0,1000)

In [29]:
model.metabolites.get_by_id('2dr5p_c').name = '2-Deoxy-D-ribose 5-phosphate'

In [30]:
#Assuming Reaction irreversible because of dG of -30.
model.reactions.DRBK.bounds = (-1000,0)

In [31]:
model.reactions.GSPMDS.bounds = (0,1000)

In [32]:
model.reactions.FTHFLi.bounds = (0,1000)

In [33]:
model.reactions.NAMNPP.bounds = (-1000,0)

In [34]:
model.reactions.RBK_L1.bounds = (0,1000)

In [35]:
model.reactions.RBFK.bounds = (0,1000)

In [36]:
model.reactions.DURIK1.bounds = (0,1000)

In [37]:
model.reactions.ATDAMh.id = 'DADK'

In [38]:
#ATP hydrolysis so should not be reversible
model.reactions.AIRLIG.bounds = (0,1000)

In [39]:
model.reactions.HPPK2.bounds = (0,1000)

In [40]:
model.metabolites.xu1p__D_c.names = 'L-Xylulose 1-phosphate'

In [41]:
#Phosphate donnor, so likely to be irreversible (and based on dG)
model.reactions.XYLUPT.bounds = (0,1000)

In [42]:
#BUTPT functions predominantly to form butanoate, so I will contrain the reaction. also the dG for this reaction is around 30.
model.reactions.BUTPT.bounds =(-1000,0)

In [43]:
model.reactions.DHBS.bounds = (0,1000)

In [44]:
model.reactions.GTPDPK.bounds = (0,1000)

In [45]:
model.reactions.ANNAT.id = 'NMNAT'

In [46]:
model.reactions.NMNAT.bounds = (0,1000)

In [47]:
model.reactions.CTPS2.bounds = (0,1000)

In [48]:
model.reactions.ASNS1.bounds = (0,1000)

In [49]:
model.reactions.TMPK.bounds = (0,1000)

In [50]:
model.reactions.DGNSK.bounds = (0,1000) 

In [51]:
model.metabolites.r5p_c.name = 'D-Ribose 5-phosphate'

In [52]:
model.metabolites.prpp_c.name = '5-Phospho-alpha-D-ribose 1-diphosphate'

In [53]:
model.reactions.R5PPT.bounds = (0,1000)

In [54]:
model.reactions.R5PPPT.bounds = (0,1000)

In [55]:
#save&commit
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')

In [5]:
solution = model.optimize()
solution

,fluxes,reduced_costs
IDPh,0.0,-7.100339e-03
CAT,0.0,8.861010e-18
PDHam1hi,0.0,1.641027e-18
CCP,0.0,4.430505e-18
HYDA,0.0,-0.000000e+00
...,...,...
EX_amylose_e,0.0,-3.381114e-02
EX_pyr_e,0.0,-2.028668e-03
FORt,0.0,-6.675205e-18
PIt,-1000.0,-0.000000e+00


## Other things to look into
- GAM in biomass reaction
- ATP synthesis: ATP synthase & respiration 
- NADK and ALDD2x rcts so they can be irreversible as they should be
- where does the high biomass then still come from?